In [2]:
import pandas as pd
import requests
import json
import datetime
import time
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, Float, Date

In [11]:
SP_fname = "../data/SP500.csv"
sp500 = pd.read_csv(SP_fname, parse_dates = ["Quarter"])

In [4]:
api_key = "QIQF06535Z8VEPZ7"

database_path = "../data/pricedata.sqlite"
engine = create_engine(f"sqlite:///{database_path}", echo=False)
columns={'monthend_date': Date, 'ticker': String(8), 'price': Float, 'wealth_index':Float}

In [14]:
ticker_list = sp500['Symbol'].tolist()[369:]
first_pass = False
for ticker in ticker_list:
    print (ticker)
    
    # Initialize the dataframe
    prices_df = pd.DataFrame(columns=['monthend_date', 'ticker', 'price', 'wealth_index'])

    url = "https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=" + ticker + "&apikey=" + api_key
    
    # Per the documentation, we con only get 5 responses per minute, or one every 12 seconds.  Let's wait 13 seconds to be safe.
    if (not first_pass):
        time.sleep(13)
    response = requests.get(url).json()
    price_data = response["Monthly Adjusted Time Series"]
    
    for daily_key, daily_value in price_data.items():
        last_trading_date = datetime.date(*(int(s) for s in daily_key.split('-')))
        price = daily_value['4. close']
        wealth_index = daily_value['5. adjusted close']
    
        #Force the monthend_date to be the last calendar day of the month, instead of the last trading day of the month
        temp_year = last_trading_date.year
        temp_month = last_trading_date.month
        if (temp_month == 12):
            monthend_date = datetime.date (temp_year, 12, 31)
        else:
            monthend_date = datetime.date (temp_year, temp_month + 1, 1) - relativedelta(days=1)
        
        new_row = {'monthend_date':monthend_date, 'ticker':ticker, 'price':price, 'wealth_index':wealth_index}
        prices_df = prices_df.append(new_row, ignore_index=True)

    if (first_pass):
        update_mode = 'replace'
        first_pass = False
    else:
        update_mode = 'append'

    prices_df.to_sql('pricedata', con=engine, dtype=columns,  if_exists=update_mode)
    

PKI
PLD
PM
PNC
PNR
PNW
PPG
PPL
PRGO
PRU
PSA
PSX
PVH
PWR
PXD
PYPL
QCOM
QRVO
RCL
RE
REG
REGN
RF
RHI
RJF
RL
RMD
ROK
ROL
ROP
ROST
RSG
RTN
SBAC
SBUX
SCHW
SEE
SHW
SIVB
SJM
SLB
SLG
SNA
SNPS
SO
SPG
SPGI
SRE
STI
STT
STX
STZ
SWK
SWKS
SYF
SYK
SYMC
SYY
T
TAP
TDG
TEL
TFX
TGT
TIF
TJX
TMO
TMUS
TPR
TRIP
TROW
TRV
TSCO
TSN
TSS
TTWO
TWTR
TXN
TXT
UA
UAL
UDR
UHS
ULTA
UNH
UNM
UNP
UPS
URI
USB
UTX
V
VAR
VFC
VIAB
VLO
VMC
VNO
VRSK
VRSN
VRTX
VTR
VZ
WAB
WAT
WBA
WCG
WDC
WEC
WELL
WFC
WHR
WLTW
WM
WMB
WMT
WRK
WU
WY
WYNN
XEC
XEL
XLNX
XOM
XRAY
XRX
XYL
YUM
ZBH
ZION
ZTS


In [15]:
engine.execute("SELECT ticker, monthend_date, price, wealth_index FROM pricedata WHERE ticker='XRAY'").fetchall()

[('XRAY', '2019-09-30', 51.13, 51.13),
 ('XRAY', '2019-08-31', 52.15, 52.15),
 ('XRAY', '2019-07-31', 54.45, 54.45),
 ('XRAY', '2019-06-30', 58.36, 58.36),
 ('XRAY', '2019-05-31', 53.87, 53.7877),
 ('XRAY', '2019-04-30', 51.13, 51.0519),
 ('XRAY', '2019-03-31', 49.59, 49.5143),
 ('XRAY', '2019-02-28', 41.76, 41.6217),
 ('XRAY', '2019-01-31', 41.95, 41.811),
 ('XRAY', '2018-12-31', 37.21, 37.0867),
 ('XRAY', '2018-11-30', 37.78, 37.5657),
 ('XRAY', '2018-10-31', 34.63, 34.4336),
 ('XRAY', '2018-09-30', 37.74, 37.526),
 ('XRAY', '2018-08-31', 39.92, 39.6018),
 ('XRAY', '2018-07-31', 48.11, 47.7265),
 ('XRAY', '2018-06-30', 43.77, 43.4211),
 ('XRAY', '2018-05-31', 43.81, 43.3738),
 ('XRAY', '2018-04-30', 50.34, 49.8387),
 ('XRAY', '2018-03-31', 50.31, 49.809),
 ('XRAY', '2018-02-28', 56.06, 55.4047),
 ('XRAY', '2018-01-31', 60.81, 60.0992),
 ('XRAY', '2017-12-31', 65.83, 65.0605),
 ('XRAY', '2017-11-30', 67.01, 66.1389),
 ('XRAY', '2017-10-31', 61.07, 60.2761),
 ('XRAY', '2017-09-30', 59.